### One Hot Encoding of Frequent Categories

We have already learned that high cardinality and rare labels may result in certain categories appearing only in the train set, therefore causing over-fitting, or only in the test set, and then our models wouldn't know how to score those observations.

We also learned in the previous lecture on one hot encoding, that if categorical variables contain multiple labels, then by re-encoding them with dummy variables we will expand the feature space dramatically.

**In order to avoid these complications, we can create dummy variables only for the most frequent categories**

This procedure is also called one hot encoding of top categories.

In fact, in the winning solution of the *KDD* 2009 cup: ["Winning the KDD Cup Orange Challenge with Ensemble Selection"](http://www.mtome.com/Publications/CiML/CiML-v3-book.pdf), the authors limit one hot encoding to the 10 most frequent labels of the variable. This means that they would make one binary variable for each of the 10 most frequent labels only.

OHE of frequent or top categories is equivalent to grouping all the remaining categories under a new category. We will have a better look at grouping rare values into a new category in a later notebook in this section.


### Advantages of OHE of top categories

- Straightforward to implement
- Does not require hrs of variable exploration
- Does not expand massively the feature space
- Suitable for linear models


### Limitations

- Does not add any information that may make the variable more predictive
- Does not keep the information of the ignored labels


Often, categorical variables show a few dominating categories while the remaining labels add little information. Therefore, OHE of top categories is a simple and useful technique.

### Note

The number of top variables is set arbitrarily. In the KDD competition the authors selected 10, but it could have been 15 or 5 as well. This number can be chosen arbitrarily or derived from data exploration.


## In this assignment:
You have to perform one hot encoding with:
- pandas and NumPy
- Feature-Engine

And the advantages and limitations of these implementations using the House Prices dataset.

In [4]:
import numpy as np
import pandas as pd

# to split the datasets
from sklearn.model_selection import train_test_split

# for one hot encoding with feature-engine
from feature_engine.encoding import OneHotEncoder

In [6]:
# load dataset

data = pd.read_csv('houseprice.csv',usecols=['Neighborhood', 'Exterior1st', 'Exterior2nd', 'SalePrice'])

data.head()

,Neighborhood,Exterior1st,Exterior2nd,SalePrice
0,CollgCr,VinylSd,VinylSd,208500
1,Veenker,MetalSd,MetalSd,181500
2,CollgCr,VinylSd,VinylSd,223500
3,Crawfor,Wd Sdng,Wd Shng,140000
4,NoRidge,VinylSd,VinylSd,250000


In [7]:
# let's have a look at how many labels each variable has

for i in data:
    print(i,':',len(data[i].unique()),'labels')

Neighborhood : 25 labels
Exterior1st : 15 labels
Exterior2nd : 16 labels
SalePrice : 663 labels


In [8]:
# let's explore the unique categories for Neighborhood, Exterior1st and Exterior2nd
data.Neighborhood.unique()

array(['CollgCr', 'Veenker', 'Crawfor', 'NoRidge', 'Mitchel', 'Somerst',
       'NWAmes', 'OldTown', 'BrkSide', 'Sawyer', 'NridgHt', 'NAmes',
       'SawyerW', 'IDOTRR', 'MeadowV', 'Edwards', 'Timber', 'Gilbert',
       'StoneBr', 'ClearCr', 'NPkVill', 'Blmngtn', 'BrDale', 'SWISU',
       'Blueste'], dtype=object)

In [9]:
data.Exterior1st.unique()

array(['VinylSd', 'MetalSd', 'Wd Sdng', 'HdBoard', 'BrkFace', 'WdShing',
       'CemntBd', 'Plywood', 'AsbShng', 'Stucco', 'BrkComm', 'AsphShn',
       'Stone', 'ImStucc', 'CBlock'], dtype=object)

In [10]:
data.Exterior2nd.unique()

array(['VinylSd', 'MetalSd', 'Wd Shng', 'HdBoard', 'Plywood', 'Wd Sdng',
       'CmentBd', 'BrkFace', 'Stucco', 'AsbShng', 'Brk Cmn', 'ImStucc',
       'AsphShn', 'Stone', 'Other', 'CBlock'], dtype=object)

### Encoding important

It is important to select the top or most frequent categories based of the train data. Then, we will use those top categories to encode the variables in the test data as well

In [18]:
# let's separate into training and testing set when target = SalePrice, and test_size = 30%

usecols=['Neighborhood','Exterior1st','Exterior2nd', 'SalePrice']
X_train, X_test, y_train, y_test = train_test_split(
    data[usecols],
    data['SalePrice'], # target
    test_size=0.3, # percentage of observations in the test set
    random_state=0) # seed for reproducibility
X_train.shape, X_test.shape

((1022, 4), (438, 4))

In [24]:
# let's first examine how OHE expands the feature space

ohe = OneHotEncoder()
ohe.fit_transform(X_train).shape

(1022, 57)

From the initial 3 categorical variables, we end up with 53 variables. 

These numbers are still not huge, and in practice we could work with them relatively easily. However, in real-life datasets, categorical variables can be highly cardinal, and with OHE we can end up with datasets with thousands of columns.


## OHE with pandas and NumPy


### Advantages

- quick
- returns pandas dataframe
- returns feature names for the dummy variables

### Limitations:

- it does not preserve information from train data to propagate to test data

In [20]:
# let's find the top 10 most frequent categories for the variable 'Neighborhood'

X_train.groupby('Neighborhood').size().sort_values(ascending=False).head(10)

Neighborhood
NAmes      151
CollgCr    105
OldTown     73
Edwards     71
Sawyer      61
Somerst     56
Gilbert     55
NridgHt     51
NWAmes      51
SawyerW     45
dtype: int64

In [26]:
# let's make a list with the most frequent (top 10) categories of the variable

list(X_train.groupby('Neighborhood').size().sort_values(ascending=False).keys()[:10])

['NAmes',
 'CollgCr',
 'OldTown',
 'Edwards',
 'Sawyer',
 'Somerst',
 'Gilbert',
 'NridgHt',
 'NWAmes',
 'SawyerW']

In [27]:
# Create 2 functions to calculate top categories and to apply OHE

def findTopCategories(data,variable):
    return list(data.groupby(variable).size().sort_values(ascending=False))
    

def applyOHE(data,top_cat):
    return OneHotEncoder(top_categories=top_cat).fit_transform(data)
    

In [28]:
# let's see the result
Result=applyOHE(X_train,10)


Result.insert(0,'Exterior2nd',X_train['Exterior2nd'])
Result.insert(0,'Exterior1st',X_train['Exterior1st'])
Result.insert(0,'Neighborhood',X_train['Neighborhood'])

Result.head()

,Neighborhood,Exterior1st,Exterior2nd,SalePrice,Neighborhood_NAmes,Neighborhood_CollgCr,Neighborhood_OldTown,Neighborhood_Edwards,Neighborhood_Sawyer,Neighborhood_Somerst,Neighborhood_Gilbert,Neighborhood_NWAmes,Neighborhood_NridgHt,Neighborhood_SawyerW,Exterior1st_VinylSd,Exterior1st_HdBoard,Exterior1st_Wd Sdng,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_CemntBd,Exterior1st_BrkFace,Exterior1st_WdShing,Exterior1st_Stucco,Exterior1st_AsbShng,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_HdBoard,Exterior2nd_MetalSd,Exterior2nd_Plywood,Exterior2nd_CmentBd,Exterior2nd_Wd Shng,Exterior2nd_BrkFace,Exterior2nd_AsbShng,Exterior2nd_Stucco
64,CollgCr,VinylSd,VinylSd,219500,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
682,ClearCr,Wd Sdng,Wd Sdng,173000,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
960,BrkSide,Wd Sdng,Plywood,116500,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1384,Edwards,WdShing,Wd Shng,105000,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
1100,SWISU,Wd Sdng,Wd Sdng,60000,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


Note how we now have 30 additional dummy variables instead of the 53 that we would have had if we had created dummies for all categories.

## One hot encoding of top categories with Feature-Engine

### Advantages

- quick
- creates the same number of features in train and test set



In [29]:
# let's separate into training and testing set

usecols=['Neighborhood','Exterior1st','Exterior2nd', 'SalePrice']
X_train, X_test, y_train, y_test = train_test_split(
    data[usecols],
    data['SalePrice'], # target
    test_size=0.3, # percentage of observations in the test set
    random_state=0) # seed for reproducibility
X_train.shape, X_test.shape

((1022, 4), (438, 4))

In [34]:
# Apply OneHotEncoder for top 10 Variables

encoder = OneHotEncoder( top_categories=10, variables=['Neighborhood','Exterior1st','Exterior2nd'], drop_last=False)


In [38]:
# this is the list of variables that the encoder will transform

encoder.fit(X_train)


train_t= encoder.transform(X_train)
test_t= encoder.transform(X_test)

encoder.variables

['Neighborhood', 'Exterior1st', 'Exterior2nd']

In [39]:
train_t.head()

,SalePrice,Neighborhood_NAmes,Neighborhood_CollgCr,Neighborhood_OldTown,Neighborhood_Edwards,Neighborhood_Sawyer,Neighborhood_Somerst,Neighborhood_Gilbert,Neighborhood_NWAmes,Neighborhood_NridgHt,Neighborhood_SawyerW,Exterior1st_VinylSd,Exterior1st_HdBoard,Exterior1st_Wd Sdng,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_CemntBd,Exterior1st_BrkFace,Exterior1st_WdShing,Exterior1st_Stucco,Exterior1st_AsbShng,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_HdBoard,Exterior2nd_MetalSd,Exterior2nd_Plywood,Exterior2nd_CmentBd,Exterior2nd_Wd Shng,Exterior2nd_BrkFace,Exterior2nd_AsbShng,Exterior2nd_Stucco
64,219500,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
682,173000,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
960,116500,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1384,105000,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
1100,60000,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


**Note**

If the argument variables is left to None, then the encoder will automatically identify **all categorical variables**. Is that not sweet?

The encoder will not encode numerical variables. So if some of your numerical variables are in fact categories, you will need to re-cast them as object before using the encoder.